In [ ]:
pwd()

- (원래 파이썬 경로) /Users/younghokim/.julia/conda/3/aarch64/bin/python

- (astropy2 경로) /Users/younghokim/anaconda3/envs/astropy2

In [ ]:
# 파이썬 환경 변수 설정

# ENV["PYTHON"] = "/Users/younghokim/anaconda3/envs/astropy2/bin/python"

In [ ]:
# using Pkg
# Pkg.build("PyCall")

In [ ]:
# using PyCall
# PyCall.python

# Imaging a Black Hole using only Closure Quantities

In [1]:
using Comrade
using Pyehtim

In [2]:
using StableRNGs
rng = StableRNG(123)

StableRNGs.LehmerRNG(state=0x000000000000000000000000000000f7)

## 1. Load the Data

In [3]:
obs = ehtim.obsdata.load_uvfits("comrade_data/SR1_M87_2017_096_lo_hops_netcal_StokesI.uvfits")

Python: <ehtim.obsdata.Obsdata object at 0x16fdd9ea0>

Now we do some minor preprocessing:

- Scan average the data since the data have been preprocessed so that the gain phases are coherent.

- Add 1% systematic noise to deal with calibration issues that cause 1% non-closing errors.

In [4]:
obs = scan_average(obs).add_fractional_noise(0.015)

[ Info: Before homogenizing we have 25 unique times
[ Info: After homogenizing we have 25 unique times


Python: <ehtim.obsdata.Obsdata object at 0x2b4ee96f0>

Now, we extract our closure quantities from the EHT data set.

In [5]:
dlcamp, dcphase  = extract_table(obs, LogClosureAmplitudes(;snrcut=3), ClosurePhases(;snrcut=3))

/Users/younghokim/.julia/environments/v1.9/.CondaPkg/env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/younghokim/.julia/environments/v1.9/.CondaPkg/env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/younghokim/.julia/environments/v1.9/.CondaPkg/env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(EHTObservation{Float64,Comrade.EHTLogClosureAmplitudeDatum{Float64}, ...}
  source: M87
  mjd: 57849
  frequency: 2.27070703125e11
  bandwidth: 1.856e9
  stations: [:AA, :AP, :AZ, :JC, :LM, :PV, :SM]
  nsamples: 128
, EHTObservation{Float64,Comrade.EHTClosurePhaseDatum{Float64}, ...}
  source: M87
  mjd: 57849
  frequency: 2.27070703125e11
  bandwidth: 1.856e9
  stations: [:AA, :AP, :AZ, :JC, :LM, :PV, :SM]
  nsamples: 152
)

In [6]:
dlcamp

EHTObservation{Float64,Comrade.EHTLogClosureAmplitudeDatum{Float64}, ...}
  source: M87
  mjd: 57849
  frequency: 2.27070703125e11
  bandwidth: 1.856e9
  stations: [:AA, :AP, :AZ, :JC, :LM, :PV, :SM]
  nsamples: 128


In [7]:
dcphase

EHTObservation{Float64,Comrade.EHTClosurePhaseDatum{Float64}, ...}
  source: M87
  mjd: 57849
  frequency: 2.27070703125e11
  bandwidth: 1.856e9
  stations: [:AA, :AP, :AZ, :JC, :LM, :PV, :SM]
  nsamples: 152


## 2. Build the Model/Posterior

이 모델에서는 점 소스의 래스터로 구성된 이미지 모델을 사용하고, 일부 펄스 또는 커널로 컨볼루션하여 연속 이미지 객체를 만드는 일반 이미지 모델을 사용할 것입니다.

In [8]:
function sky(θ, metadata)
    (;fg, c, σimg) = θ
    (;K, meanpr, grid, cache) = metadata
    
    # Construct the image model we fix the flux to 0.6 Jy in this case
    cp = meanpr .+ σimg.*c.params
    rast = ((1-fg))*K(to_simplex(CenteredLR(), cp))
    img = IntensityMap(rast, grid)
    m = ContinuousImage(img, cache)
    
    # Add a large-scale gaussian to deal with the over-resolved mas flux
    g = modify(Gaussian(), Stretch(μas2rad(250.0), μas2rad(250.0)), Renormalize(fg))
    return m + g
end

sky (generic function with 1 method)

이제 이미지 모델을 설정해 보겠습니다. EHT의 공칭 해상도는 20-25μas입니다. 또한 EHT는 더 큰 시야각에 그다지 민감하지 않으며, 일반적으로 60~80μas면 M87의 콤팩트 플럭스를 설명하기에 충분합니다. 이 점을 고려하면 이미지를 설명하는 데 적은 수의 픽셀만 사용하면 됩니다.

In [9]:
npix = 32
fovxy = μas2rad(150.0)

7.27220521664304e-10

이제 배열 정보를 입력하여 캐시를 구성할 수 있습니다.

In [10]:
grid = imagepixels(fovxy, fovxy, npix, npix)

2-element GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}:
 LinRange{Float64}(-3.5224744018114725e-10, 3.5224744018114725e-10, 32)
 LinRange{Float64}(-3.5224744018114725e-10, 3.5224744018114725e-10, 32)

In [11]:
grid

2-element GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}:
 LinRange{Float64}(-3.5224744018114725e-10, 3.5224744018114725e-10, 32)
 LinRange{Float64}(-3.5224744018114725e-10, 3.5224744018114725e-10, 32)

In [12]:
buffer = IntensityMap(zeros(npix, npix), grid)

2-dimensional KeyedArray(NamedDimsArray(...)) with keys:
↓   X ∈ 32-element LinRange{Float64,...}
→   Y ∈ 32-element LinRange{Float64,...}
And data, 32×32 NamedDimsArray(::Matrix{Float64}, (:X, :Y)):
                 (-3.52247e-10)  …  (3.29522e-10)  (3.52247e-10)
 (-3.52247e-10)     0.0               0.0            0.0
 (-3.29522e-10)     0.0               0.0            0.0
 (-3.06796e-10)     0.0               0.0            0.0
 (-2.84071e-10)     0.0               0.0            0.0
 (-2.61345e-10)     0.0          …    0.0            0.0
 (-2.38619e-10)     0.0               0.0            0.0
 (-2.15894e-10)     0.0               0.0            0.0
 (-1.93168e-10)     0.0               0.0            0.0
 (-1.70442e-10)     0.0               0.0            0.0
    ⋮                            ⋱    ⋮            
  (1.47717e-10)     0.0               0.0            0.0
  (1.70442e-10)     0.0               0.0            0.0
  (1.93168e-10)     0.0               0.0            0.0

In [13]:
buffer.X

32-element LinRange{Float64, Int64}:
 -3.52247e-10, -3.29522e-10, -3.06796e-10, …, 3.29522e-10, 3.52247e-10

In [14]:
buffer.Y

32-element LinRange{Float64, Int64}:
 -3.52247e-10, -3.29522e-10, -3.06796e-10, …, 3.29522e-10, 3.52247e-10

In [15]:
nfft_result = NFFTAlg(dlcamp)

VLBISkyModels.ObservedNUFT{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Matrix{Float64}}(NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}(1, 4, 2.0, :kaiser_bessel, AbstractNFFTs.POLYNOMIAL, true, false, true, 0x00000000), [-4.405690154666661e9 787577.6145833326 … -5.999801315555549e9 -15551.297851562484; -4.523017159111106e9 -1.6838098888888871e6 … 3.059254300444441e9 118294.64453124987])

In [16]:
nfft_result.alg

NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}(1, 4, 2.0, :kaiser_bessel, AbstractNFFTs.POLYNOMIAL, true, false, true, 0x00000000)

In [17]:
nfft_result.uv

2×274 Matrix{Float64}:
 -4.40569e9   7.87578e5  -4.4443e9   …  -5.9998e9   -15551.3
 -4.52302e9  -1.68381e6  -4.59783e9      3.05925e9       1.18295e5

```
VLBISkyModels.NUFTCache{VLBISkyModels.ObservedNUFT{NFFTAlg{Float64,
                                                            AbstractNFFTs.PrecomputeFlags,
                                                            UInt32},
                                                    Matrix{Float64}
                                                   }, 
                        NFFT.NFFTPlan{Float64, 2, 1}, 
                        Vector{ComplexF64}, 
                        BSplinePulse{3}, 
                        KeyedArray{Float64, 
                                        2, 
                                    NamedDimsArray{(:X, :Y), 
                                                    Float64, 
                                                    2, 
                                                    Matrix{Float64}
                                                   }, 
                                    GriddedKeys{(:X, :Y), 
                                                Tuple{LinRange{Float64, Int64},
                                                        LinRange{Float64, Int64}
                                                      }, 
                                                ComradeBase.NoHeader, 
                                                Float64}
                                                }
                                    }
                        
(VLBISkyModels.ObservedNUFT{NFFTAlg{Float64,
                                    AbstractNFFTs.PrecomputeFlags,
                                    UInt32},
                            Matrix{Float64}}

(NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}

(1, 4, 2.0, :kaiser_bessel, AbstractNFFTs.POLYNOMIAL, true, false, true, 0x00000000),

[-4.405690154666661e9 787577.6145833326 … -5.999801315555549e9 -15551.297851562484;
 -4.523017159111106e9 -1.6838098888888871e6 … 3.059254300444441e9 118294.64453124987]),

NFFTPlan with 274 sampling points for an input array of size(32, 32) and an output array of size(274,) with dims 1:2,

ComplexF64[0.7014670924816703 - 0.519511453845776im,
0.9999999862106403 - 6.398624051637031e-5im,
0.6947985314160843 - 0.5233374146500747im,
0.9328845732113544 - 0.16336912604935844im,
0.9329421134238738 + 0.16331744240563614im,
0.6948519110447875 + 0.5233083088067162im,
0.7925669922789302 + 0.40478845348863063im,
0.9999999864389528 - 6.519719206334107e-5im,
0.9340555032742773 - 0.1520910238460368im,
0.6896791530194316 - 0.5261728587493705im  …
0.8376666586657548 + 0.1784956664688869im,
0.9196690775195503 + 0.09173305645579018im,
0.9135584686897926 + 0.29199716871629755im,
0.9135610039654034 + 0.2919905620278203im,
0.8899638273254515 + 0.28719490128071723im,
0.9568100109264257 + 0.09814170760184636im,
0.8899622135777404 + 0.287201492222944im,
0.9925668796456906 + 0.0027765725149323534im,
0.8376650833595861 - 0.17848899851834446im,
0.9999999999247222 + 7.335331210317892e-6im],

BSplinePulse{3}(),

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])
```

In [20]:
cache = create_cache(nfft_result.alg, buffer, BSplinePulse{3}())

VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}(NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}(1, 4, 2.0, :kaiser_bessel, AbstractNFFTs.POLYNOMIAL, true, false, true, 0x00000000), nothing, nothing, BSplinePulse{3}(), [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [21]:
using VLBIImagePriors, Distributions, DistributionsAD

In [22]:
fwhmfac = 2*sqrt(2*log(2))
mpr = modify(Gaussian(), Stretch(μas2rad(50.0)./fwhmfac))
imgpr = intensitymap(mpr, grid)

2-dimensional KeyedArray(NamedDimsArray(...)) with keys:
↓   X ∈ 32-element LinRange{Float64,...}
→   Y ∈ 32-element LinRange{Float64,...}
And data, 32×32 NamedDimsArray(::Matrix{Float64}, (:X, :Y)):
                 (-3.52247e-10)  …  (3.29522e-10)  (3.52247e-10)
 (-3.52247e-10)     6.37537e-8        1.32434e-7     6.37537e-8
 (-3.29522e-10)     1.32434e-7        2.751e-7       1.32434e-7
 (-3.06796e-10)     2.62014e-7        5.44273e-7     2.62014e-7
 (-2.84071e-10)     4.93724e-7        1.0256e-6      4.93724e-7
 (-2.61345e-10)     8.86092e-7   …    1.84065e-6     8.86092e-7
 (-2.38619e-10)     1.51463e-6        3.14629e-6     1.51463e-6
 (-2.15894e-10)     2.46586e-6        5.12225e-6     2.46586e-6
 (-1.93168e-10)     3.82352e-6        7.94248e-6     3.82352e-6
 (-1.70442e-10)     5.64668e-6        1.17297e-5     5.64668e-6
    ⋮                            ⋱    ⋮            
  (1.47717e-10)     7.94248e-6        1.64987e-5     7.94248e-6
  (1.70442e-10)     5.64668e-6        1.172

이제 실제로 심플렉스에서 이미지를 모델링하고 있으므로 평균 이미지에 단위 플럭스가 있는지 확인해야 합니다.

In [23]:
imgpr ./= flux(imgpr)

meanpr = to_real(CenteredLR(), Comrade.baseimage(imgpr))
metadata = (;meanpr,K=CenterImage(imgpr), grid, cache)

(meanpr = [-7.554221225633777 -6.823167558636959 … -6.823167558636959 -7.554221225633777; -6.823167558636959 -6.0921138916401425 … -6.0921138916401425 -6.823167558636959; … ; -6.823167558636959 -6.0921138916401425 … -6.0921138916401425 -6.823167558636959; -7.554221225633777 -6.823167558636959 … -6.823167558636959 -7.554221225633777], K = CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}([0.9944957386363638 -0.005326704545454383 … 0.00532670454545453 0.005504261363636347; -0.005326704545454383 0.9948393969941346 … 0.0051606030058651 0.005326704545454565; … ; 0.00532670454545453 0.0051606030058651 … 0.9948393969941348 -0.005326704545454544; 0.005504261363636347 0.005326704545454565 … -0.005326704545454544 0.9944957386363638], (32, 32)), grid = GriddedKeys{(:X, :Y)}
	X: LinRange{Float64}(-3.5224744018114725e-10, 3.5224744018114725e-10, 32)
	Y: LinRange{Float64}(-3.5224744018114725e-10, 3.5224744018114725e-10, 32)
, cache = VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeF

또한 이미지의 해상도에 대한 합리적인 추측이 필요합니다. 전파 천문학의 경우 이미지에서 대략적으로 가장 긴 기준선이 이 값을 제공합니다. 이를 픽셀 공간에 넣으려면 픽셀 크기로 나누면 됩니다.

In [24]:
beam = beamsize(dlcamp)
rat = (beam/(step(grid.X)))

5.326336637737519

In [33]:
beam

1.2104441588297418e-10

In [34]:
rat

5.326336637737519

가우스 마르코프 랜덤 필드를 효율적으로 만들기 위해 먼저 이미지 픽셀 수에 따라 선형적으로 스케일링할 수 있는 여러 가지 양을 미리 계산합니다. 이렇게 하면 일반적인 가우스 프로세스에서 얻을 수 있는 N^3 스케일링을 크게 개선할 수 있습니다.

In [25]:
crcache = MarkovRandomFieldCache(meanpr);

베이지안 접근법의 장점 중 하나는 기존 RML 평가와 달리 prior/레귤레이터의 하이퍼파라미터에 맞출 수 있다는 것입니다. 이 계층적 선행을 구축하기 위해 먼저 정규화기 하이퍼파라미터를 받아들이고 해당 하이퍼파라미터가 주어진 이미지 선행을 반환하는 맵을 만들 것입니다.

In [26]:
fmap = let crcache=crcache
    x -> GaussMarkovRandomField(x, 1.0, crcache)
end

#3 (generic function with 1 method)

이제 마침내 이미지 prior를 형성할 수 있습니다. 이를 위해 과적합을 방지하기 위해 상관 관계 길이가 역 감마 선형에 의해 주어지는 계층적 선형을 사용합니다. 가우스 마르코프 랜덤 필드는 매우 유연한 모델입니다. 과적합을 방지하기 위해 복잡성에 불이익을 주는 전제를 사용하는 것이 일반적입니다. 따라서 우리는 평균 이미지와의 유사성을 강화하고 평활성을 선호하는 전제 조건을 사용하고자 합니다.

In [27]:
cprior = HierarchicalPrior(fmap, InverseGamma(1.0, -log(0.01*rat)))

prior = NamedDist(c = cprior, σimg = truncated(Normal(0.0, 0.1); lower = 0.0), fg=Uniform(0.0, 1.0))

lklhd = RadioLikelihood(sky, dlcamp, dcphase;
                        skymeta = metadata)
post = Posterior(lklhd, prior)

Posterior{RadioLikelihood{Comrade.ModelMetadata{typeof(sky), NamedTuple{(:meanpr, :K, :grid, :cache), Tuple{Matrix{Float64}, CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}, VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}}}}, Nothing, Tuple{Comrade.EHTObservation{Float64, Comrade.EHTLogClosureAmplitudeDatum{Float64}, StructArrays.StructVector{Comrade.EHTLogClosureAmplitudeDatum{Float64}, NamedTuple{(:measurement, :error, :U1, :V1, :U2, :V2, :U3, :V3, :U4, :V4, :T, :F, :quadrangle), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float6

In [37]:
cprior

HierarchicalPrior{var"#3#4"{MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}}}, InverseGamma{Float64}}(
priormap: #3
hyperprior: InverseGamma{Float64}(
invd: Gamma{Float64}(α=1.0, θ=0.3410052124381751)
θ: 2.932506494109095
)

)


In [39]:
post

Posterior{RadioLikelihood{Comrade.ModelMetadata{typeof(sky), NamedTuple{(:meanpr, :K, :grid, :cache), Tuple{Matrix{Float64}, CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}, VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}}}}, Nothing, Tuple{Comrade.EHTObservation{Float64, Comrade.EHTLogClosureAmplitudeDatum{Float64}, StructArrays.StructVector{Comrade.EHTLogClosureAmplitudeDatum{Float64}, NamedTuple{(:measurement, :error, :U1, :V1, :U2, :V2, :U3, :V3, :U4, :V4, :T, :F, :quadrangle), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float6

## 3. Reconstructing the Image

In [28]:
post.prior

NamedDist{(:c, :σimg, :fg), Tuple{HierarchicalPrior{var"#3#4"{MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}}}, InverseGamma{Float64}}, Truncated{Normal{Float64}, Continuous, Float64, Float64, Nothing}, Uniform{Float64}}}(
dists: (HierarchicalPrior{var"#3#4"{MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}}}, InverseGamma{Float64}}(
priormap: #3
hyperprior: InverseGamma{Float64}(
invd: Gamma{Float64}(α=1.0, θ=0.3410052124381751)
θ: 2.932506494109095
)

)
, Truncated(Normal{Float64}(μ=0.0, σ=0.1); lower=0.0), Uniform{Float64}(a=0.0, b=1.0))
)


NamedDist{(:c, :σimg, :fg), 
          Tuple{HierarchicalPrior{var"#3#4"{MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, 
                LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}}}, InverseGamma{Float64}},
                Truncated{Normal{Float64}, Continuous, Float64, Float64, Nothing}, 
                            Uniform{Float64}}}(
                                dists: (HierarchicalPrior{var"#3#4"{MarkovRandomFieldCache{
                                            SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, 
                                            Matrix{Float64}}}, InverseGamma{Float64}}(
priormap: #3
hyperprior: InverseGamma{Float64}(
invd: Gamma{Float64}(α=1.0, θ=0.3410052124381751)
θ: 2.932506494109095
)

)
, Truncated(Normal{Float64}(μ=0.0, σ=0.1); lower=0.0), Uniform{Float64}(a=0.0, b=1.0))
)

## 4. Using GaussianRandomFields

이 posterior에서 샘플링하려면 먼저 제약된 파라미터 공간에서 제약되지 않은 공간으로 이동하는 것이 편리합니다(즉, 변환된 후방의 지지대가 (-∞, ∞) 인 경우). 이 작업은 asflat 함수를 사용하여 수행됩니다.

In [32]:
tpost = asflat(post.prior)

LoadError: MethodError: no method matching GaussMarkovRandomField(::Float64, ::Float64, ::MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}})

[0mClosest candidates are:
[0m  GaussMarkovRandomField(::T, ::C) where {T<:Number, C}
[0m[90m   @[39m [32mVLBIImagePriors[39m [90m~/.julia/packages/VLBIImagePriors/7f9ze/src/markovrf/[39m[90m[4mgmrf.jl:37[24m[39m
[0m  GaussMarkovRandomField(::Number, [91m::MarkovRandomFieldCache[39m)
[0m[90m   @[39m [32mVLBIImagePriors[39m [90m~/.julia/packages/VLBIImagePriors/7f9ze/src/markovrf/[39m[90m[4mgmrf.jl:84[24m[39m
[0m  GaussMarkovRandomField(::Number, [91m::Tuple{Int64, Int64}[39m)
[0m[90m   @[39m [32mVLBIImagePriors[39m [90m~/.julia/packages/VLBIImagePriors/7f9ze/src/markovrf/[39m[90m[4mgmrf.jl:72[24m[39m
[0m  ...


In [ ]:
ndim = dimension(tpost)

In [ ]:
tpost

In [ ]:
using ComradeOptimization
using OptimizationOptimJL
using Zygote
f = OptimizationFunction(tpost, Optimization.AutoZygote())
prob = Optimization.OptimizationProblem(f, prior_sample(rng, tpost), nothing)
sol = solve(prob, LBFGS(); maxiters=5_00);

In [ ]:
xopt = transform(tpost, sol)

In [ ]:
using Plots
residual(skymodel(post, xopt), dlcamp, ylabel="Log Closure Amplitude Res.")

In [ ]:
residual(skymodel(post, xopt), dcphase, ylabel="|Closure Phase Res.|")

In [ ]:
import WGLMakie as CM
img = intensitymap(skymodel(post, xopt), μas2rad(150.0), μas2rad(150.0), 100, 100)
CM.image(img, axis=(xreversed=true, aspect=1, title="MAP Image"), colormap=:afmhot, figure=(;resolution=(400, 400),))

In [ ]:
using ComradeAHMC
using Zygote
metric = DiagEuclideanMetric(ndim)
chain, stats = sample(post, AHMC(;metric, autodiff=Val(:Zygote)), 700; nadapts=500, init_params=xopt)

In [ ]:
msamples = skymodel.(Ref(post), chain[501:2:end]);

In [ ]:
using StatsBase
imgs = intensitymap.(msamples, μas2rad(150.0), μas2rad(150.0), 128, 128)
mimg = mean(imgs)
simg = std(imgs)
fig = CM.Figure(;resolution=(400, 400));
CM.image(fig[1,1], mimg,
                   axis=(xreversed=true, aspect=1, title="Mean Image"),
                   colormap=:afmhot)
CM.image(fig[1,2], simg./(max.(mimg, 1e-5)),
                   axis=(xreversed=true, aspect=1, title="1/SNR",), colorrange=(0.0, 2.0),
                   colormap=:afmhot)
CM.image(fig[2,1], imgs[1],
                   axis=(xreversed=true, aspect=1,title="Draw 1"),
                   colormap=:afmhot)
CM.image(fig[2,2], imgs[end],
                   axis=(xreversed=true, aspect=1,title="Draw 2"),
                   colormap=:afmhot)
fig

In [ ]:
p = plot();
for s in sample(chain[501:end], 10)
    residual!(p, vlbimodel(post, s), dlcamp)
end
ylabel!("Log-Closure Amplitude Res.");
p

In [ ]:
p = plot();
for s in sample(chain[501:end], 10)
    residual!(p, vlbimodel(post, s), dcphase)
end
ylabel!("|Closure Phase Res.|");
p